In [1]:
# !pip install selenium
# !pip install tqdm
# !pip install beautifulsoup4
# !pip install pyautogui
# !pip install pandas
# !pip install webdriver-manager

import time
import os
import pyautogui as pag
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
# table 정도 한번에 불러오는 html_table_paser의 make2d 기능 test

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import pandas as pd

from selenium import webdriver # webdriver를 이용해 해당 브라우저를 열기 위해
from selenium.webdriver import ActionChains # 일련의 작업들을(ex.아이디 입력, 비밀번호 입력, 로그인 버튼 클릭...) 연속적으로 실행할 수 있게 하기 위해
from selenium.webdriver.common.keys import Keys # 키보드 입력을 할 수 있게 하기 위해
from selenium.webdriver.common.by import By # html요소 탐색을 할 수 있게 하기 위해
from selenium.webdriver.support.ui import WebDriverWait # 브라우저의 응답을 기다릴 수 있게 하기 위해
from selenium.webdriver.support import expected_conditions as EC # html요소의 상태를 체크할 수 있게 하기 위해
# 이 외에도 필요한 모듈이 있다면 따로 호출해준다.

In [2]:
#카테고리 페이지 실행
url = "https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001&gateCd=Drawer&t_page=%EB%93%9C%EB%A1%9C%EC%9A%B0_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%ED%83%AD_%EB%8C%80%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&t_1st_category_type=%EB%8C%80_%EC%8A%A4%ED%82%A8%EC%BC%80%EC%96%B4"
driver = webdriver.Chrome()
driver.get(url)

In [34]:
#전체 크롤링 진행
for cos in range(1, 4):
    #품목창 들어가기
    review_text = driver.find_element(By.CSS_SELECTOR, f'#Contents > div > div.ct-menu > ul > li:nth-child({cos}) > a > span').text
    review = driver.find_element(By.CSS_SELECTOR, f'#Contents > div > div.ct-menu > ul > li:nth-child({cos}) > a > span').click()
    print(f"현재는 {review_text} 입니다.")

    #화장품 품목별 개수 확인
    cos_list = driver.find_element(By.CSS_SELECTOR, "#Contents > ul.cate_list_box").text
    cos_lists = list(cos_list.split('\n'))

    for skincare1 in range(2, len(cos_lists)):
        #항목으로 들어갑니다.
        select_skin = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul.cate_list_box > li:nth-child({skincare1}) > a').click()
        time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
        
        #현재 화장품 종류 확인_이 변수를 데이터 셋에 넣어주면 됩니다.
        cosmetics = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul.cate_list_box > li:nth-child({skincare1}) > a').text
        print(f"현재 항목은 {cosmetics}입니다.")

        #48개씩 보기
        select_show = driver.find_element(By.CSS_SELECTOR, '#Contents > div.cate_align_box > div.count_sort.tx_num > ul > li:nth-child(3) > a').click()
        time.sleep(2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
        print('항목을 48개씩 보겠습니다.')

        #페이지 리스트 만들기
        next_pages = []
        pre_pages = []
        page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing').text
        pages = list(page.split())
        if '다음' in pages: #다음 페이지 버튼이 있으면 확인합니다.
                select_page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing > a.next').click()
                time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
                
                #다음 페이지의 리스트를 만듭니다.
                next_page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing').text
                next_pages = list(next_page.split())
                next_pages = next_pages[3:]
                driver.back() 
        pre_pages = pages[:10]
        pages = pre_pages + next_pages #다음장의 페이지 항목을 더해줍니다.(최대 20페이지가 넘지 않는다는 상황에서 작동합니다.)
        print(f'페이지 항목은 {pages}입니다.')

        for now_page in pages: #페이지 항목 수 만큼 반복하여 진행합니다.
            now_pages = int(now_page)
            if now_pages==1 or 10 or 11:  #1,11번 페이지는 다른 주소명을 갖고 있어 pass로 넘겼습니다. 
                pass

            elif now_pages > 10:
                select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
                time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

            else:   #다음 페이지로 넘어가는 코드입니다.
                select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
                time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

            print(f'현재는 {now_pages}페이지 입니다.')
            for row in range(7, 19): #총 12개의 행입니다.
                for col in range(1, 5): #총 4개의 열입니다.
                    if col == 1: #첫번째 상품은 다른 형식이라 따로 지정
                        product = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({row}) > li.flag > div > a > img').click()
                        reviews = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').click()
                        time.sleep(5) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
                        
                        #상품이 클릭된 페이지 입니다. 여기서 상품 정보를 긁어오는 코드를 넣으시면 됩니다.

                        print(f"{row-6}행에 {col}열의 항목이 스크랩되었습니다.")
                        driver.back()
                    else:
                        product = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({row}) > li:nth-child({col}) > div > a > img').click()
                        reviews = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').click()
                        time.sleep(5) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
                        
                        #상품이 클릭된 페이지 입니다. 여기서 상품 정보를 긁어오는 코드를 넣으시면 됩니다.

                        print(f"{row-6}행에 {col}열의 항목이 스크랩되었습니다.")
                        driver.back()
            
            if now_pages == 10: #10페이지가 끝나면 다음 페이지로 넘어갑니다.
                select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
                time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

    #초기 페이지로 돌아가기 위해서 시도합니다.
    start_page = driver.find_element(By.CSS_SELECTOR, '#btnGnbOpen').click()
    start_page = driver.find_element(By.CSS_SELECTOR, '#gnbAllMenu > ul > li:nth-child(1) > div:nth-child(2) > p:nth-child(1) > a').click()
    print('작업을 완료하고 초기 페이지로 돌아왔습니다.')
    

현재는 토너/로션/올인원 입니다.
현재 항목은 스킨/토너입니다.
항목을 48개씩 보겠습니다.
페이지 항목은 ['1', '2', '3', '4', '5', '6', '7', '8']입니다.
현재는 1페이지 입니다.
1행에 1열의 항목이 스크랩되었습니다.
1행에 2열의 항목이 스크랩되었습니다.
1행에 3열의 항목이 스크랩되었습니다.
1행에 4열의 항목이 스크랩되었습니다.
2행에 1열의 항목이 스크랩되었습니다.
2행에 2열의 항목이 스크랩되었습니다.
2행에 3열의 항목이 스크랩되었습니다.
2행에 4열의 항목이 스크랩되었습니다.
3행에 1열의 항목이 스크랩되었습니다.
3행에 2열의 항목이 스크랩되었습니다.
3행에 3열의 항목이 스크랩되었습니다.
3행에 4열의 항목이 스크랩되었습니다.


KeyboardInterrupt: 

In [37]:
#피처 추출 코드
#내용이 어느 부분인지 잘 모르겠어서 진행하지 못했습니다. 나머지 부탁드릴게요!
name = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name").text
brand = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_brand").text
price = int(driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong").text.replace(",", ""))
sale_price = int(driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong").text.replace(",", ""))
date = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.date').text
rate = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.review_point > span').text
id = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.info > div > p.info_user > a.id').text
skin_type = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(2) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
# purchase_type_2 = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(9) > div.info > div > div > div:nth-child(1) > a').text
select_1_title = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span').text
select_1_content = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
select_2_title = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span').text
select_2_content = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
select_3_title = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span').text
select_3_content = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text
txt = driver.find_element(By.CSS_SELECTOR,'#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner').text

In [38]:
name, brand, price, sale_price, date, rate, id ,skin_type

('[단독기획] 넘버즈인 3번 결광가득 에센스 토너 300ml 대용량 기획',
 '넘버즈인',
 24000,
 24000,
 '2023.07.20',
 '5점만점에 5점',
 '요니양',
 '건성에 좋아요')